# TP 1 -  AI for OSINT

De nombreuse informations sur un profil sont aujourd'hui disponibles sur internet grâce aux réseaux sociaux. Ces informations peuvent permettre à un potentiel attaquant d'identifier les cibles les plus pertinentes pour lancer une attaque de spear phishing ou faire du social engineering.

Cependant, l'investigation en ligne (OSINT) est coûteuse en temps humain : les cibles sont de plus en plus nombreuses et la quantité d'information disponible explose. Les méthodes d'IA permettent aux attaquants d'automatiser cette première phase de reconnaissance

# *Clustering Twitter Users*

Pour trouver les cibles les plus susceptible de mordre à l'hameçon, la première étape de la pipeline consiste en un algorithme de clustering qui place un utilisateur donné dans la catégorie "High Value Target" ou non. 

Dans ce TP, nous allons nous intéresser aux jeux olympiques de Tokyo 2020.

## Outline

Ce TP est divisé en 5 tâches,  : 
- Récupérer et comprendre les données utilisées
- Définir les caractéristiques à extraire (feature engineering)
- Sélection et paramétrisation d'un modèle
- Évaluation des performances
- Itération des modèles

## Mise en place de l'environnement

Avant de commencer, commencez par créer et activer un environnement virtuel Python qui servira pour ce TP :
```
$ python3 -m venv .venv
$ source .venv/bin/activate
```

Installez ensuite les dépendances nécessaire :
````
$ pip install -r requirements.py
````


Définissez l'environnement virtuel comme kernel jupiter, puis exécutez la cellul suivante pour importer les librairies qui nous seront utiles.

In [1]:
import os
import csv

## Récupération et compréhension des données

Nous allons travailler avec des données plubliquement accessibles sur Twitter. 

Nous verrons dans un premier temps comment mettre en place un client pour requêter l'API de twitter et récupérer nous-même les données. 

Ensuite, nous utiliserons un jeu de données déjà construit, sur les JO de Tokyo. Elles ont été collectées à l'aide de la librairie tweepy pour accéder à l'API de Twitter, en utilisant hashtag pertinent pour le sujet (#Tokyo2020)

### Mise en place du client

Vous pouvez créer un compte Twitter, et remplir ci-dessous vos credentials. 

In [4]:
from twikit.client import Client

client = Client(language='en-US')
client.login(auth_info_1='username',
             password='pwd')
client.save_cookies('cookies.json')


In [5]:
client.load_cookies(path='cookies.json')
user = client.get_user_by_screen_name()
print(user.name)


KeyError: 'result'

### Récupération des données

1. Définir une requête pertinente pour les données qui nous intéressent, et utiliser les mots-clés `since:YYYY-MM-DD` et `until:YYYY-MM-DD` pour cadrer la date.

In [6]:
query = ''

2. Écrire une fonction `get_tweets` qui permette de récupérer les tweets correspondant à la requête définie ci-dessus, et de sauvegarder les informations qui nous intéressent dans un fichier csv local. On limitera ici le nombre de tweets téléchargés à 500.
    
    *Indice : utilisez la fonction `client.search_tweets()`de `twikit` pour requêter l'API. Explorez dans un premier temps les métadonnées qui peuvent être disponibles pour un tweet*

    Attention ! L'API a une rate-limit de 500 tweets toutes les 15 minutes

In [ ]:
tweet = client.search_tweet(query = query, product='Top', count=10)

In [ ]:
filepath=os.path.join('data', 'twikit_tokyo_2020_tweets.csv')

columns = []

def get_tweets(client, query, filepath=filepath, max_results=500):
    tweets = [] 
    results = client.search_tweet(query = query, product='Top', count=max_results)
    tweets+=results
    while True:
        tweets += results.next()
        if len(tweets)>= max_results:
            break
    with open(filepath, 'w', newline='') as f:
        writer = csv.writer(f, delimiter = ',')
        writer.writerow(columns)
        for tweet in tweets:
            writer.writerow(...)

    return 

3. Utiliser `get_tweets()`pour récupérer les tweets qui nous intéressent, et vérifier brièvement que les résultats correspondent à nos attentes.

In [ ]:
client.load_cookies(path='cookies.json')
get_tweets(client, query)

In [ ]:
len(tweets)

In [ ]:
tweets[0].text

Pour aller plus vite, dans la suite du TD, nous utiliserons tous le même jeu de données sur les JO de Tokyo en 2020.
Le fichier CSV contenant les colonnes qui nous intéresse est disponible dans le dossier `data`.

### Visualisation des données

Maintenant que nous avons collecté les données, il est important de les connaître et de les comprendre : sont-elles pertinentes ? Y a-t-il du bruit ? Peut-on le filtrer ? Toutes ces questions nécessitent un inspection minutieuse des résultats obtenus. 

3. La bibliothèque la plus couramment utilisée pour le traitement de données en Data Science est `pandas`(ref to pandas). Utiliser cet outil pour créer un dataframe contenant les colonnes `'date'`, `'text'` et `'user_name'`.  
    
    *Indice : la fonction `to_datetime()` est utile pour manipuler plus facilement les données temporelles*

In [ ]:
import pandas as pd

In [ ]:
tweet_df = ...

In [ ]:
tweet_df.head()

4. Utiliser les fonctionnalités des dataframes pandas pour visualiser le nombre de tweets par jour de notre ensemble de données

5. Et le nombre de tweets par utilisateur ?

Avec cet échantillon, faisons un peu de traitement de texte brut pour examiner le contenu des tweets. Une première façon simple - et souvent instructive - d'inspecter le contenu des données textuelles consiste à examiner les n-grammes les plus courants. En modélisation linguistique, un "n-gramme" est une collection contiguë de quelque n éléments - il s'agit souvent de mots séparés par des espaces. Par exemple, les 3-grammes de la phrase "Le chien a mangé mes devoirs" seraient "le chien a", "chien a mangé", "a mangé mes", "mangé mes devoirs".

Nous allons donc afficher les 3-grammes les plus fréquents, afin de se faire une idée des données textuelles dont nous disposons. Si nous voyons un contenu qui ne nous semble pas pertinent, nous pouvons revenir en arrière et modifier notre requête.


6. Calculez les n-grammes les plus courants 

    *Indice : la bibliothèque nltk peut être utile pour tokenizer (séparer en mots) le texte*

In [ ]:
from nltk.tokenize.casual import TweetTokenizer
from nltk.util import everygrams
from collections import Counter

In [ ]:
def get_all_tokens(tweet_df):
    """
    Helper function to generate a list of text tokens from concatenating
    all of the text contained in Tweets 
    """
    # concat entire corpus
    all_text = ' '.join((txt for txt in tweet_df['text'] if type(txt) == str))
    # tokenize
    tokens = (TweetTokenizer(preserve_case=False,
                            reduce_len=True,
                            strip_handles=False)
              .tokenize(all_text))
    # remove symbol-only tokens for now
    tokens = [tok for tok in tokens if not tok in string.punctuation]
    return tokens

In [ ]:
tokens = get_all_tokens(tweet_df)

In [ ]:
# calculate a range of ngrams using some handy functions
top_grams = Counter(everygrams(tokens, min_len=2, max_len=4))

top_grams.most_common(25)

7. Quelles informations peut-on retrouver sur les JO de Tokyo en explorant les données ? 

## Feature Engineering

Le but, ici, est de passer de données semi-structurées (csv) à la matrice de données bidimensionnelle d'observations et de caractéristiques attendue par de nombreuses bibliothèques de ML.

L''objectif de ce TD est de rassembler utilisateurs de Twitter en groupes de similarité. De nombreuses informations peuvent être disponibles via l'API de Twitter, pour plus de simplicité nous nous concentrerons sur la description des utilisateurs. 

### Preprocessing

1. Les liens URL ne sont pas porteurs de beaucoup d'informations (raccourcis par twitter). Commençons par enlever les URL du texte et les remplacer par un pattern unique, `"lien_url"`

Une étape importante du traitement de texte consiste à diviser la string en tokens (ou mots). Il existe de nombreuses façons de découper une chaîne de texte en tokens, mais dans le cadre de cette discussion, nous nous intéresserons principalement à l'anglais. Dans ce cas, le découpage du texte via les espaces reste la manière la plus simple de procéder. 

Nous pouvons également choisir de créer notre propre tokenizer explicite si les données (et la tâche) l'exigent. Une méthode particulière qui fonctionne avec les données Twitter est le TweetTokenizer de NLTK. Il préserve les symboles @ et # au début des mots, et peut également "réduire" les caractères répétés - c'est-à-dire que lolll, lollllll, et lollllllllllll se réduiront tous à la même représentation "lolll" (trois "l"). 

2. Créer une fonction `my_tokenizer()` qui sépare du texte en tokens.

3. Enlever les "stop-words" (mots les plus courants, qui ne sont pas vraiment porteurs d'information - 'the', etc.)

    *Indice : utiliser des listes de stop-words déjà faites, comme celle de `python-stop-words`. Avez-vous pense à tous les langages pertinents ?*

### Vectorisation

La plupart des algorithmes de ML prêts à l'emploi, par exemple ceux de sklearn, attendent une entrée sous forme de matrice de données bidimensionnelle de valeurs numériques : observations (lignes) x caractéristiques (colonnes). Pour créer une représentation numérique de données textuelles, nous devons vectoriser les caractéristiques du texte (tokens).

1. Qu'est-ce que le TF-IDF d'un mot dans un corpus de texte ? 

2. Créer un TF-IDF grâce à sklearn, en tirant parti des arguments `preprocessor`, `stop-words`, `tokenizer` et `max-features`

3. Commentez le résultat : pourquoi la matrice est si sparse (clairsemée) ? 

4. Quelles autres features peut-on ajouter à nos données ? 

## Sélection d'un modèle et de ses hyperparamètres

De nombeaux modèles de clustering sont disponibles via la bibliothèque `scikit-learn`([https://scikit-learn.org/stable/modules/clustering.html])

### K-Means

KMeans est un choix courant car il est très rapide pour des quantités modérées de données. Comme la plupart des algorithmes de ML, KMeans a des hyperparamètres qui doivent être choisis de manière appropriée. Dans le cas présent, ce paramètre est k, le nombre de cluster dans nos données.

En non supervisé, il n'est pas facile de calculer (et d'optimiser) un score de précision. Nous devons donc utiliser d'autres techniques pour comparer les modèles entre eux afin de sélectionner k. Comme nous ne connaissons pas ce nombre a priori, une technique consiste à comparer la valeur d'une certaine métrique de qualité parmi une gamme de k potentiels. Il existe un certain nombre de mesures de qualité connues, dont nous n'utiliserons que deux : le silhouette score et l'inertie.

1. Que représente le silhouette score ? Et l'inertie ?

2. En utilisant `scikit-learn`, calculer le silhouette-score ainsi que l'inertie pour plusieurs valeurs de k différentes : [2, 50, 200, 500] (cette étape peut prendre quelques minutes)

3. En utilisant `matplotlib.pyplot`, afficher l'évolution du silhouette-score et de l'inertie en fonction de la valeur de k

4. Analyser les résultats ci-dessus : quelle serait la meilleure valeur de k ? Comment pourrait-on affiner cette recherche ?

5. Entraîner le modèle, avec les bonnes valeurs de k, sur l'ensemble du jeu de données

## Résultats

1. Combien y a-t-il d'utilisateurs dans chaque cluster ? Que peut-on en déduire ?

Rappelons que les utilisateurs sont représentés dans un espace de paramètres comprenant les mots utilisés dans leur biographie. Dans l'algorithme KMeans, la représentation résultante de ces cluster est constituée des coordonnées du centroïde de chaque cluster dans cet espace de tokens. Ainsi, une autre façon d'examiner nos résultats est de poser la question suivante : pour chaque centroïde de cluster, quels sont les tokens qui ont la plus grande projection sur ce centroïde ? En d'autres termes, quels sont les tokens les plus fortement associés à chaque groupe ?

2. Afficher les n features les plus importantes pour chaque cluster

3. Pour un cluster donné, afficher quelques biographies et analyser les résultats. 

4. (optionnel) : utiliser `MulticoreTSNE`pour résuire la dimension de la représentation des utilisateurs, et représenter en deux dimensions les cluster avec les fonction suivante : 

In [ ]:
def get_plottable_df(users, bios, two_d_coords, labels):
    """
    Combine the necessary pieces of data to create a data structure that plays
    nicely with the our 2d tsne chart.

    Note: assumes that all argument data series
    are in the same order e.g. the first user, bio, coords, and label
    all correspond to the same user.
    """
    # set up color palette
    num_labels = len(set(labels))
    colors = sns.color_palette('hls', num_labels).as_hex()
    color_lookup = {v:k for k,v in zip(colors, set(labels))}
    # combine data into a single df
    df = pd.DataFrame({'uid': users,
                       'text': bios,
                       'label': labels,
                       'x_val': two_d_coords[:,0],
                       'y_val': two_d_coords[:,1],
                      })
    # convert labels to colors
    df['color'] = list(map(lambda x: color_lookup[x], labels))
    return df

def plot_tsne(df, title='t-SNE plot'):
    # add our DataFrame as a ColumnDataSource for Bokeh
    plot_data = ColumnDataSource(df)
    # configure the chart
    tsne_plot = figure(title=title, plot_width=800, plot_height=700, tools=('pan, box_zoom, reset'))
    # add a hover tool to display words on roll-over
    tsne_plot.add_tools(
        HoverTool(tooltips = """<div style="width: 400px;">(@label) @text</div>""")
    )
    # draw the words as circles on the plot
    tsne_plot.circle('x_val', 'y_val',
                     source=plot_data,
                     color='color',
                     line_alpha=0.2,
                     fill_alpha=0.1,
                     size=7,
                     hover_line_color='black')
    # configure visual elements of the plot
    tsne_plot.title.text_font_size = '12pt'
    tsne_plot.xaxis.visible = False
    tsne_plot.yaxis.visible = False
    tsne_plot.grid.grid_line_color = None
    tsne_plot.outline_line_color = None
    return tsne_plot

## Itération

1. Le choix d'un modèle dépend beaucoup des hypothèses faites sur la répartition des données initiales. Quel serait le modèle le plus pertinent selon vous ? Vous pouvez vous référer à [la page d'acceuil scikit learn](https://scikit-learn.org/stable/modules/clustering.html). Réitérer le processus ci dessus avec le modèle choisi.

2. Quelles améliorations pourrait-on faire sur le traitement des données ? 